In [2]:
import pandas as pd
import numpy as np
import statistics as stat
import random as random
import math as math

# Ear Phenotypes

In [3]:
useCols = [1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
df1 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
df2 = pd.read_csv('../data/earphenotypesformatted.csv', na_values = {'a/n', 'a'}, usecols = useCols)
fileList = [df1, df2]
traitList = ['Ear Width', 'Kernel Fill Length', 'Kernel Row Number', 'Kernels per Row', 'Ear Weight', 'Kernel Count', 'Cob Length', 'Cob Width', 'Cob Weight', '100 Kernel weight']

In [4]:
# What if we just concat and pivot
ears = pd.concat([df1, df2])
ears = ears.reset_index(drop=False)
ears = ears.replace({'22.94.':'22.94'})
ears['Kernel Mass'] = ears['Ear Weight'].astype(float) - ears['Cob Weight'].astype(float)
ears['QR Code'].astype(str)
ears['QR Code'] = ears['QR Code'].str.upper()
ears['earNum'] = ears.groupby('QR Code').cumcount()
ears = ears[ears['earNum'] < 4]
plots = pd.pivot(ears, index = 'QR Code', columns = 'earNum')
traitList.append('Kernel Mass')
plots = plots.astype(float)

In [5]:
plots.dtypes

                    earNum
index               0         float64
                    1         float64
                    2         float64
                    3         float64
Ear Width           0         float64
                    1         float64
                    2         float64
                    3         float64
Kernel Fill Length  0         float64
                    1         float64
                    2         float64
                    3         float64
Kernel Row Number   0         float64
                    1         float64
                    2         float64
                    3         float64
Kernels per Row     0         float64
                    1         float64
                    2         float64
                    3         float64
Ear Weight          0         float64
                    1         float64
                    2         float64
                    3         float64
Kernel Count        0         float64
                    1  

In [ ]:
# Now that we have our nice data frame, we need to calculate the mean and standard error with all possible sets of 4 and all possible sets of 2 for each trait.
index30 = [1, 2, 3]
index31 = [0, 2, 3]
index32 = [0, 1, 2]
index20 = [0, 1]
index21 = [0, 2]
index22= [0, 3]
index23 = [1, 2]
index24= [1, 3]
index25= [2, 3]
index3List = [index30, index31, index32]
index3ListStr = ['30', '31', '32']
index2List = [index20, index21, index22, index23, index24, index25]
index2ListStr = ['20', '21', '22', '23', '24', '25']

results = pd.DataFrame()
for i in traitList:
    mean = i + 'mean' + '4'
    se = i + 'se' + '4'
    meanSeries = (plots[i][[0, 1, 2, 3]]).mean(axis = 1)
    results[mean] = meanSeries
    seSeries = (plots[i][[0, 1, 2, 3]]).sem(axis = 1)
    results[se] = seSeries
    
    for j in range(3):
        mean = i + 'mean' + index3ListStr[j]
        se = i + 'se' + index3ListStr[j]
        index = index3List[j]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries
        
    for k in range(6):
        mean = i + 'mean' + index2ListStr[k]
        se = i + 'se' + index2ListStr[k]
        index = index2List[k]
        meanSeries = (plots[i][index]).mean(axis = 1)
        seSeries = (plots[i][index]).sem(axis = 1)
        results[mean] = meanSeries
        results[se] = seSeries

In [60]:
meanSeries

QR Code
 SCOTTSBLUFF$HYBRID-HIPS - LOW NITROGEN$REP1$PLOT1134$ROW21$RANGE11$LH198 X PHZ51              121.10
6240$ROW33$RANGE13$2369 X 3IIH6                                                                103.55
NORTH PLATTE$HIPS - FULL IRRIGATION - BORDER NITROGEN$REP1$PLOT35$ROW3$RANGE12$LH74 X PHM49    132.90
NORTH PLATTE$HIPS - FULL IRRIGATION - HIGH NITROGEN$REP1$PLOT369$ROW1$RANGE26$LH195 X LH185    174.98
NORTH PLATTE$HIPS - FULL IRRIGATION - HIGH NITROGEN$REP1$PLOT370$ROW2$RANGE26$PHK76 X W606S    126.75
                                                                                                ...  
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1336$ROW11$RANGE18$PHK76 X LH82              85.45
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1337$ROW11$RANGE19$PHK76 X LH198            116.48
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1338$ROW11$RANGE20$LH195 X LH123HT           85.34
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1339$ROW11$RANGE21$PHP0

In [61]:
results

,Ear Widthmean4,Ear Widthse4,Ear Widthmean30,Ear Widthse30,Ear Widthmean31,Ear Widthse31,Ear Widthmean32,Ear Widthse32,Ear Widthmean20,Ear Widthse20,...,Kernel Massmean21,Kernel Massse21,Kernel Massmean22,Kernel Massse22,Kernel Massmean23,Kernel Massse23,Kernel Massmean24,Kernel Massse24,Kernel Massmean25,Kernel Massse25
QR Code,,,,,,,,,,,,,,,,,,,,,
SCOTTSBLUFF$HYBRID-HIPS - LOW NITROGEN$REP1$PLOT1134$ROW21$RANGE11$LH198 X PHZ51,3.375,0.239357,3.500000,0.288675,3.500000,0.288675,3.333333,0.333333,3.00,0.00,...,142.975,4.165,125.265,21.875,107.860,30.950,90.150,13.240,121.10,17.71
6240$ROW33$RANGE13$2369 X 3IIH6,3.500,0.288675,3.666667,0.333333,3.666667,0.333333,3.333333,0.333333,3.00,0.00,...,53.460,45.320,58.230,50.090,70.290,28.490,75.060,33.260,103.55,4.77
NORTH PLATTE$HIPS - FULL IRRIGATION - BORDER NITROGEN$REP1$PLOT35$ROW3$RANGE12$LH74 X PHM49,4.125,0.125000,4.000000,0.000000,4.166667,0.166667,4.166667,0.166667,4.25,0.25,...,136.500,24.800,157.700,3.600,117.650,5.950,138.850,15.250,132.90,21.20
NORTH PLATTE$HIPS - FULL IRRIGATION - HIGH NITROGEN$REP1$PLOT369$ROW1$RANGE26$LH195 X LH185,4.375,0.125000,4.333333,0.166667,4.500000,0.000000,4.333333,0.166667,4.25,0.25,...,180.875,11.495,163.485,5.895,160.530,31.840,143.140,14.450,174.98,17.39
NORTH PLATTE$HIPS - FULL IRRIGATION - HIGH NITROGEN$REP1$PLOT370$ROW2$RANGE26$PHK76 X W606S,3.875,0.125000,4.000000,0.000000,3.833333,0.166667,3.833333,0.166667,3.75,0.25,...,128.940,18.520,108.230,2.190,136.675,10.785,115.965,9.925,126.75,20.71
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1336$ROW11$RANGE18$PHK76 X LH82,4.250,0.144338,4.333333,0.166667,4.166667,0.166667,4.166667,0.166667,4.25,0.25,...,78.900,0.200,85.650,6.550,85.000,6.300,91.750,0.450,85.45,6.75
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1337$ROW11$RANGE19$PHK76 X LH198,4.500,0.000000,4.500000,0.000000,4.500000,0.000000,4.500000,0.000000,4.50,0.00,...,143.755,22.915,139.395,27.275,129.760,8.920,125.400,13.280,116.48,4.36
SCOTTSBLUFF$HYBRID-HIPS - MEDIUM NITROGEN$REP2$PLOT1338$ROW11$RANGE20$LH195 X LH123HT,4.250,0.144338,4.166667,0.166667,4.166667,0.166667,4.333333,0.166667,4.50,0.00,...,124.995,23.495,108.835,39.655,115.145,13.645,98.985,29.805,85.34,16.16


In [65]:
avgSE = pd.DataFrame()
for t in traitList:
    traitSE2 = t + '.SE.2'
    traitSE3 = t + '.SE.3'
    
    seCols2 = []
    for i in index2ListStr:
        colName = t + 'se' + i
        seCols2.append(colName)
     
    seCols3 = []
    for j in index3ListStr:
        colName = t + 'se' + j
        seCols3.append(colName)
        avgSE[traitSE2] = results[seCols2].mean(axis = 1)
        avgSE[traitSE3] = results[seCols3].mean(axis = 1)
    

In [ ]:
avgSE

In [71]:
corrPerSet = pd.DataFrame()
for t in traitList:
    fullMeanCol = t + 'mean' + '4'
    
    for i in index2ListStr:
        colName = t + 'mean' + i
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet[colName] = pd.Series()
        corrPerSet[colName].append(cor)
    for j in index3ListStr:
        colName = t + 'mean' + i
        cor = results[colName].corr(results[fullMeanCol])
        corrPerSet[colName].append(cor)
    

AttributeError: 'Series' object has no attribute 'append'

In [70]:
corrPerSet

,Ear Widthmean20,Ear Widthmean21,Ear Widthmean22,Ear Widthmean23,Ear Widthmean24,Ear Widthmean25,Kernel Fill Lengthmean20,Kernel Fill Lengthmean21,Kernel Fill Lengthmean22,Kernel Fill Lengthmean23,...,100 Kernel weightmean22,100 Kernel weightmean23,100 Kernel weightmean24,100 Kernel weightmean25,Kernel Massmean20,Kernel Massmean21,Kernel Massmean22,Kernel Massmean23,Kernel Massmean24,Kernel Massmean25
